In [27]:
# 3D Local IFS fractal self-similarity and encoder/decoder for T1 MRI

using Images
using PyPlot
using FileIO

testfile = "JeffT1.tiff"
t1img = load(testfile)

192×192×128 Array{Gray{N0f8},3}:
[:, :, 1] =
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 ⋮                              

In [2]:
# splitblocks: given an image t1img, returns array of all blocks size s^3
function splitblocks(t1img,s)
    vblocks = []
    pxstride = n -> Int(s*(n-1)+1)
    dims = size(t1img)
    N = dims[1]/s # expect square image
    M = dims[3]/s # 
    for i = 1:N
        y = pxstride(i)
        for j = 1:N
            x = pxstride(j)
            for k = 1:M
                z = pxstride(k)
                block = t1img[y:y+s-1,x:x+s-1,z:z+s-1]
                push!(vblocks,block)
            end
        end
    end
    return vblocks
end

# downsample: given a NxNxN voxel cube, downsamples by a half
function downsample(vblock)
    N = size(vblock)[1]
    M = Int(N/2)
    downblock = zeros(M,M,M)
    for i = 1:M
        y = Int(2*(i-1)+1)
        for j = 1:M
            x = Int(2*(j-1)+1)
            for k = 1:M
                z = Int(2*(k-1)+1)
                downblock[i,j,k] = mean(vblock[y:y+1,x:x+1,z:z+1])
            end
        end
    end
    return downblock
end

# alphabeta: given two voxel cubes, minimizes root mean square error
# R range block, D domain block (should already be downsampled)
function greyscaleaffine(R,D)
    A = zeros(2,2)
    A[1,1] = sum(D.^2)
    A[1,2] = A[2,1] = sum(D)
    A[2,2] = length(D)
    b = zeros(2,1)
    b[1] = sum(R.*D)
    b[2] = sum(R)
    alpha, beta = A\b
    return alpha, beta
end

greyscaleaffine (generic function with 1 method)

In [ ]:
function selfsimilarity(t1img)
    tic()
    rblocks = splitblocks(t1img,4) #pick block sizes, domain = 2*range
    dblocks = splitblocks(t1img,8)

    errs = []
    zcount = [] # keep track of the number of empty blocks we need not encode
    for i = 1:length(rblocks)
        if (i % 100 == 0) println(i) end
        Ri = rblocks[i]
        if sum(Ri) == 0
    #         push!(zcount, i)
            continue
        else
            for j = 1:length(dblocks)
                Dj = dblocks[j]
                if sum(Dj) == 0
                    continue
                else
                    # Purely translational, dblocks are 8x8x8
#                     err = Ri - Dj
                    
                    # Two-scale translational
#                     Dj = downsample(Dj)
#                     err = Ri - Dj
                    
                    # Two-scale translational with greyscale beta shift
                    # (impose stationary condition on RMSE to find beta)
#                     Dj = downsample(Dj)
#                     beta = mean(Ri-Dj)
#                     err = Ri - Dj - beta

                    # Two-scale translational with greyscale alpha scale, beta shift
                    # (impose stationary condition on RMSE to setup alpha,beta system)
                    Dj = downsample(Dj)
                    alpha,beta = greyscaleaffine(Ri,Dj)
                    err = Ri - alpha*Dj - beta

                    err = sqrt(sum(err.^2)/length(err))
                    push!(errs,err)
                end
            end        
        end
    end
    toc()
    return errs
end

# errs = selfsimilarity(t1img)
PyPlot.plt[:hist](errs,100)
PyPlot.plt[:xlabel]("Error")
PyPlot.plt[:ylabel]("Number of blocks")
PyPlot.plt[:title]("JeffT1: two-scale translational self-similarity")
ax = PyPlot.plt[:gca]()
ax[:set_xlim]((0,1))

In [7]:
# encode: given 3D image, splits into 8x8x8 i range blocks and 16x16x16 j domain blocks
# j,alpha,beta are stored such for each i, the RMSE is minimized:
# sqrt(sum((Ri-alpha(Dj)-beta)^2)/length(Ri))
# rs: rangeblock size, domainblock size is twice that
function encode(t1img, rs)
    rblocks = splitblocks(t1img,rs)
    dblocks = splitblocks(t1img,2rs)

    map!(downsample, dblocks)
    code = Array{Tuple{Int64,Int64,Float16,Float16}}(length(rblocks),1)
    for i = 1:length(rblocks)
        if (i % 1000 == 0) println(i) end
        Ri = rblocks[i]
        besterr=1; bestj=0; bestalpha=0; bestbeta=0
        # if range block is all black, skip search since can be encoded with all zeros
        if sum(Ri) != 0
            for j = 1:length(dblocks)
                Dj = dblocks[j]
                if sum(Dj) == 0
                    continue
                else
                    alpha,beta = greyscaleaffine(Ri,Dj)
                    err = Ri - alpha*Dj - beta
                    err = sqrt(sum(err.^2)/length(err))
                    if err < besterr
                        besterr = err
                        bestj = j
                        bestalpha = alpha
                        bestbeta = beta
                    end
                end
            end
        end
        code[i] = (Int64(i),Int64(bestj),Float16(bestalpha),Float16(bestbeta))
    end
    
    dims = size(t1img)
    return code, dims
end

# rasteryxz: given raster position (i.e. position in rblocks array)
# returns block indices where dim(y,x,z) = (N,M,K) the number of blocks
function rasteryxz(i,rs,blockdims)
    N,M,K = blockdims
    y = Int(div(i-1,M*K)+1)
    x = Int(div(mod(i-1,M*K),K)+1)
    z = Int((i-1)%K+1)
    y = rs*(y-1)+1
    x = rs*(x-1)+1
    z = rs*(z-1)+1
    return y,x,z
end

# constructrblocks: given range block array, rangeblock size and image dimensions
# assemble range blocks into 3D image
function constructrblocks(rblocks,rs,dims)
    f = zeros(dims)
    N = dims[1]/rs
    M = dims[3]/rs
    for i = 1:length(rblocks)
        y,x,z = rasteryxz(i,rs,(N,N,M))
        f[y:y+rs-1,x:x+rs-1,z:z+rs-1] = rblocks[i]
    end
    return f
end

# fractaltransform: 
function fractaltransform(f,code,dims,rs)
    rblocks = splitblocks(f,rs)
    dblocks = splitblocks(f,2rs)
    map!(downsample, dblocks)
    for i = 1:length(code)
        i,j,alpha,beta = code[i]
        if j == 0
            rblocks[i][:] = 0
        else
            Dj = dblocks[j]
            rblocks[i] = alpha*Dj+beta
        end
    end
    Tf = constructrblocks(rblocks,rs,dims)
end


function decode(code,dims,rs)
    f = zeros(dims) #initial image
    tol = 1e-10
    err = 1
    t = 0
    while err > tol
        t = t+1; println(t)
        Tf = fractaltransform(f,code,dims,rs)
        err = Tf-f
        err = sqrt(sum(err.^2)/length(err))
        f = Tf
    end
    println(t)
    return f
end

decode (generic function with 1 method)

In [8]:
function runencoder(rs)
    tic()
    # 2364 seconds (4-8 encoding)
    # 235 seconds (8-16 encoding)
    # 23 seconds (16-32 encoding)
    code,dims = encode(t1img,rs)
    toc()
    tic()
    # 45 iterations, 46 seconds
    # 28 iterations, 13 seconds
    # 21 iterations, 13 seconds
    g = decode(code,dims,rs)
    toc()
    # RMSE: 0.03376704489797993
    # RMSE: 0.6089250803946707
    # RMSE: 0.627663365311277
    save("testcode.tiff", map(clamp01nan, g))
    return code,dims
end

# code,dims = runencoder(4)
# g = load("testcode.tiff")
# err = t1img-g
# err = sqrt(sum(err.^2)/length(err))

1000


2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
elapsed time: 2687.81164506 seconds
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
45
elapsed time: 46.482991208 seconds


(
Tuple{Int64,Int64,Float16,Float16}[(1,0,0.0,0.0); (2,0,0.0,0.0); … ; (73727,0,0.0,0.0); (73728,0,0.0,0.0)],

(192,192,128))

In [12]:
# writefif: write fractal image format file
function writefif(code,dims)
    codestring = c -> string(c[1]," ",c[2]," ",c[3]," ",c[4],"\n")
    open("./testcode.fif", "w") do f
        write(f, string(dims[1]," ",dims[2]," ",dims[3],"\n"))
        for i = 1:length(code)
            write(f, codestring(code[i]))
        end
    end
end

writefif(code,dims)

In [35]:
#   ffmpeg -f image2 -framerate 16 -i %06d.jpg -s 384x192 ../test.avi

# writevids: compare cross sections of t1img with approximation f
f = load("testcode.tiff")
function writevids(t1img,f)
    function writeaxisvid(t1img,f,a) #a is axis number y=1,x=2,z=3
        for i = 1:size(f)[a]
            if a == 1
                fr1 = t1img[i,:,:]
                fr2 = f[i,:,:]
            elseif a == 2
                fr1 = t1img[:,i,:]
                fr2 = f[:,i,:]
            else
                fr1 = t1img[:,:,i]
                fr2 = f[:,:,i]
            end
            fr = [fr1 fr2]
            tlabel = lpad(string(i),6,"0")
            subd = string(a)
            save("temp/$subd/$tlabel.png", map(clamp01nan, fr))
        end
    end
    writeaxisvid(t1img,f,1)
    writeaxisvid(t1img,f,2)
    writeaxisvid(t1img,f,3)
end
writevids(t1img,f)

Gray{Float64}(1.7309488745098042e6)


In [19]:
b = rand(2,2)

2×2 Array{Float64,2}:
 0.46212   0.276861
 0.271685  0.801279

In [20]:
[a b]

2×4 Array{Float64,2}:
 0.38776   0.540845  0.46212   0.276861
 0.980326  0.492436  0.271685  0.801279